# motion_repr_learning/ae/utils/flags.py

In [1]:
"""
This module contrains all the flags for the motion representation learning repository
"""
from __future__ import division
import os
from os.path import join as pjoin

import tensorflow as tf

os.environ['HOME'] = 'H:\\minjae\\Trying Obama Dataset'

# Modify this function to set your home directory for this repo
def home_out(path):
    return pjoin(os.environ['HOME'], 'tmp', 'MoCap', path)

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')# added, no error


"""  							Fine-tuning Parameters 				"""

#                       Flags about the sequence processing

flags.DEFINE_integer('chunk_length', 1, 'Length of the chunks, for the data processing.')

#                               Flags about training
flags.DEFINE_float('learning_rate', 0.0001,
                   'learning rate for training .')
flags.DEFINE_float('pretraining_learning_rate', 0.001 ,
                   'learning rate for training .')

flags.DEFINE_float('variance_of_noise', 0.05, 'Coefficient for the gaussian noise '
                                              'added to every point in input during the training')

flags.DEFINE_boolean('pretrain', False,' Whether we pretrain the model in a layerwise way')
flags.DEFINE_boolean('restore', False,' Whether we restore the model from the checkpoint')

flags.DEFINE_boolean('evaluate', False, ' Whether we are evaluating the system')

flags.DEFINE_float('dropout', 0.9, 'Probability to keep the neuron on')

flags.DEFINE_integer('batch_size', 128,
                     'Size of the mini batch')

flags.DEFINE_integer('training_epochs', 20,
                     "Number of training epochs for pretraining layers")
flags.DEFINE_integer('pretraining_epochs', 5,
                     "Number of training epochs for pretraining layers")

flags.DEFINE_float('weight_decay', 0.5, ' Whether we apply weight decay')

flags.DEFINE_boolean('early_stopping', True, ' Whether we do early stopping')
flags.DEFINE_float('delta_for_early_stopping', 0.5, 'How much worst the results must get in order'
                                                    ' for training to be terminated.'
                                                    ' 0.05 mean 5% worst than best we had.')

#                       Network Architecture Specific Flags
# flags.DEFINE_integer('frame_size', 384, 'Dimensionality of the input for a single frame')
flags.DEFINE_integer('frame_size', 204, 'Dimensionality of the input for a single frame')

flags.DEFINE_integer("num_hidden_layers", 1, "Number of hidden layers")
flags.DEFINE_integer("middle_layer", 1, "Number of hidden layers")

# flags.DEFINE_integer('layer1_width', 312, 'Number of units in each hidden layer ')
flags.DEFINE_integer('layer1_width', 180, 'Number of units in each hidden layer ')

# flags.DEFINE_integer('layer2_width', 248, 'Number of units in each hidden layer ')
# flags.DEFINE_integer('layer3_width', 312, 'Number of units in each hidden layer ')

#                           Constants

flags.DEFINE_integer('seed', 123456, 'Random seed')

flags.DEFINE_string('summary_dir', home_out('summaries_exp'),
                    'Directory to put the summary data')

flags.DEFINE_string('chkpt_dir', home_out('chkpts_exp'),
                    'Directory to put the model checkpoints')

flags.DEFINE_string('results_file', home_out('results.txt'),
                    'File to put the experimental results')


# motion_repr_learning/ae/train.py

In [2]:
import time
class DataInfo(object):
    """Information about the datasets

     Will be passed to the network for creating corresponding variables in the graph
    """

    def __init__(self, data_sigma, train_shape, eval_shape, max_val, mean_pose):
        """DataInfo initializer

        Args:
          data_sigma:   variance in the dataset
          train_shape:  dimensionality of the train dataset
          eval_shape:   dimensionality of the evaluation dataset
        """
        self.data_sigma = data_sigma
        self.train_shape = train_shape
        self.eval_shape = eval_shape
        self.max_val = max_val
        self.mean_pose = mean_pose


###############################################
####                                    #######
####              TRAIN                 #######
####                                    #######
###############################################


def learning(data, data_info, just_restore=False):
    """ Training of the network

    Args:
        data:           dataset to train on
        data_info :     meta information about this dataset (such as variance, mean pose, etc.)
                        it is an object from the class DataInfo (defined at the top of this file)
        just_restore:   weather we are going to only restore the model from the checkpoint
                        or are we going to train it as well

    Returns:
        nn:             Neural Network trained on a data provided
    """

    test = False
    debug = False

    with tf.Graph().as_default():

        tf.set_random_seed(FLAGS.seed)

        start_time = time.time()

        # Read the flags
        variance = FLAGS.variance_of_noise
        num_hidden = FLAGS.num_hidden_layers # 1
        dropout = FLAGS.dropout
        learning_rate = FLAGS.learning_rate
        batch_size = FLAGS.batch_size

        hidden_shapes = [FLAGS.layer1_width
                         for j in range(num_hidden)]
        print("hidden_shapes:", hidden_shapes)
        
        # Check if the flags makes sence
        if dropout < 0 or variance < 0:
            print('ERROR! Have got negative values in the flags!')
            exit(1)

        # Allow TensorFlow to change device allocation when needed
        config = tf.ConfigProto(allow_soft_placement=True)  # log_device_placement=True)
        # Adjust configuration so that multiple executions are possible
        config.gpu_options.allow_growth = True

        # Start a session
        sess = tf.Session(config=config)

        if debug:
            sess = tf_debug.TensorBoardDebugWrapperSession(sess, "taras-All-Series:6064")

        # Create a neural network
        shape = [FLAGS.frame_size * FLAGS.chunk_length] + hidden_shapes + [
            FLAGS.frame_size * FLAGS.chunk_length]
        print("FLAGS.frame_size:", FLAGS.frame_size)
        print("shape:", shape)
        nn = DAE(shape, sess, variance, data_info)
        print('\nDAE with the following shape was created : ', shape)

        # Initialize input_producer
        sess.run(tf.local_variables_initializer())

        max_val = nn.max_val

        with tf.variable_scope("Train"):

            ##############        DEFINE  Optimizer and training OPERATOR      ############

            # Define the optimizer
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

            # Do gradient clipping
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(nn._loss, tvars), 1e12)
            train_op = optimizer.apply_gradients(zip(grads, tvars),
                                                 global_step=tf.train.get_or_create_global_step())

            # Prepare for making a summary for TensorBoard
            train_error = tf.placeholder(dtype=tf.float32, shape=(), name='train_error')
            eval_error = tf.placeholder(dtype=tf.float32, shape=(), name='eval_error')

            train_summary_op = tf.summary.scalar('Train_error', train_error)
            eval_summary_op = tf.summary.scalar('Validation_error', eval_error)

            summary_dir = FLAGS.summary_dir
            summary_writer = tf.summary.FileWriter(summary_dir, graph=tf.get_default_graph())

            num_batches = int(data.train.num_sequences / batch_size)

            # Initialize the part of the graph with the input data
            sess.run(nn._train_data.initializer,
                     feed_dict={nn._train_data_initializer: data.train.sequences})
            sess.run(nn._valid_data.initializer,
                     feed_dict={nn._valid_data_initializer: data.test.sequences})

            # Start input enqueue threads.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            if FLAGS.pretrain:
                layers_amount = len(nn.shape) - 2

                # create an optimizers
                pretrain_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

                # Make an array of the trainers for all the layers
                trainers = [pretrain_optimizer.minimize(
                    loss_reconstruction(nn.run_less_layers(nn._input_, i+1),
                                           nn.run_less_layers(nn._input_, i+1, is_target=True),
                                           max_val, pretrain=True),
                    global_step=tf.train.get_or_create_global_step(),
                    name='Layer_wise_optimizer_'+str(i))
                            for i in range(layers_amount)]

                # Initialize all the variables
                sess.run(tf.global_variables_initializer())

            else:
                print("Initializing variables ...\n")
                sess.run(tf.global_variables_initializer())

            # Create a saver
            saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
            chkpt_file = FLAGS.chkpt_dir + '/chkpt-final'

            # restore model, if needed
            if FLAGS.restore:
                saver.restore(sess, chkpt_file)
                print("Model restored from the file " + str(chkpt_file) + '.')

            if just_restore:
                coord.request_stop()
                return nn

            # A few initialization for the early stopping
            delta = FLAGS.delta_for_early_stopping  # error tolerance for early stopping
            best_error = 10000
            num_valid_batches = int(data.test.num_sequences / batch_size)

            try:  # running enqueue threads.

                # Pretrain
                if FLAGS.pretrain:
                    layerwise_pretrain(nn, trainers, layers_amount, num_batches)

                # Train the whole network jointly
                step = 0
                print('\nFinetune the whole network on ', num_batches, ' batches with ', batch_size,
                      ' training examples in each for', FLAGS.training_epochs, ' epochs...')
                print("")
                print(" ______________ ______")
                print("|     Epoch    | RMSE |")
                print("|------------  |------|")

                while not coord.should_stop():
                    _, train_error_ = sess.run([train_op, nn._reconstruction_loss], feed_dict={})

                    if step % num_batches == 0:
                        epoch = step * 1.0 / num_batches

                        train_summary = sess.run(train_summary_op, feed_dict={
                            train_error: np.sqrt(train_error_)})

                        # Print results of screen
                        epoch_str = "| {0:3.0f} ".format(epoch)[:5]
                        perc_str = "({0:3.2f}".format(epoch*100.0 / FLAGS.training_epochs)[:5]
                        error_str = "%) |{0:5.2f}".format(train_error_)[:10] + "|"
                        print(epoch_str, perc_str, error_str)

                        if epoch % 5 == 0 and test:

                            rmse = test(nn, FLAGS.data_dir + '/test_1.binary')
                            print("\nOur RMSE for the first test sequence is : ", rmse)

                            rmse = test(nn, FLAGS.data_dir + '/test_2.binary')
                            print("\nOur RMSE for the second test sequenceis : ", rmse)

                        if epoch > 0:
                            summary_writer.add_summary(train_summary, step)

                            # Evaluate on the validation sequences
                            error_sum = 0
                            for valid_batch in range(num_valid_batches):
                                curr_err = sess.run([nn._valid_loss], feed_dict={})
                                error_sum += curr_err[0]
                            new_error = error_sum / (num_valid_batches)
                            eval_sum = sess.run(eval_summary_op,
                                                feed_dict={eval_error: np.sqrt(new_error)})
                            summary_writer.add_summary(eval_sum, step)

                            # Early stopping
                            if FLAGS.early_stopping:
                                if (new_error - best_error) / best_error > delta:
                                    print('After ' + str(step) + ' steps started overfitting')
                                    break
                                if new_error < best_error:
                                    best_error = new_error

                                    # Saver for the model
                                    save_path = saver.save(sess, chkpt_file)

                            if epoch % 5 == 0:
                                # Save for the model
                                save_path = saver.save(sess, chkpt_file)
                                print('Done training for %d epochs' % (epoch))
                                print("The model was saved in file: %s" % save_path)

                    step += 1

            except tf.errors.OutOfRangeError:
                if not FLAGS.early_stopping:
                    # Save the model
                    save_path = saver.save(sess, chkpt_file)
                print('Done training for %d epochs, %d steps.' % (FLAGS.training_epochs, step))
                print("The final model was saved in file: %s" % save_path)
            finally:
                # When done, ask the threads to stop.
                coord.request_stop()

            # Wait for threads to finish.
            coord.join(threads)

        duration = (time.time() - start_time) / 60  # in minutes, instead of seconds

        print("The training was running for %.3f  min" % (duration))

        return nn

def decode(nn, represent_vec):
    """ Decoding a representation from AE (AutoEncoder)

      Args:
          nn:              trained AutoEncoder
          represent_vec:   input sequence to be encoded

      Returns:
          output_seq:  vector of encoding
    """

    print("Decoding ...")

    with nn.session.graph.as_default() as sess:

        # Obtain important constants

        sess = nn.session
        mean_pose = nn.mean_pose
        max_val = nn.max_val

        #                    GET THE DATA

        # Check if we can cut sequence into the chunks of length ae.sequence_length
        if represent_vec.shape[0] < nn.sequence_length:
            mupliplication_factor = int(nn.batch_size * nn.sequence_length /
                                        represent_vec.shape[0]) + 1

            # Pad the sequence with itself in order to fill the sequence completely
            represent_vec = np.tile(represent_vec, (mupliplication_factor, 1))
            print("Test sequence was way too short, so we padded it with itself!")

        # Split it into chunks
        all_chunks = represent_vec

        if all_chunks.shape[0] < nn.batch_size:
            mupliplication_factor = int(nn.batch_size / all_chunks.shape[0]) + 1

            # Pad the sequence with itself in order to fill the batch completely
            all_chunks = np.tile(all_chunks, (mupliplication_factor, 1))

        # Batch those chunks
        batches = np.array([all_chunks[i:i + nn.batch_size, :]
                            for i in range(0, len(all_chunks) - nn.batch_size + 1, nn.batch_size)])

        numb_of_batches = batches.shape[0]

        #                    RUN THE NETWORK

        output_batches = np.array([])

        # Go over all batches one by one
        for batch_numb in range(numb_of_batches):
            output_batch = sess.run([nn._decode],
                                    feed_dict={nn._representation: batches[batch_numb]})
            output_batches = np.append(output_batches, output_batch, axis=0) \
                if output_batches.size else np.array(output_batch)

        # Postprocess...
        output_vec = np.reshape(output_batches, (-1, FLAGS.chunk_length * FLAGS.frame_size))

        # Convert back to original values
        reconstructed = convert_back_to_3D(output_vec, max_val, mean_pose)
        print("reconstructed:", reconstructed)
        return reconstructed

###############################################
####                                    #######
####      LAYERWISE PRETRAINING         #######
####                                    #######
###############################################

# motion_repr_learning/ae/DAE.py

In [3]:
def add_noise(x, variance_multiplier, sigma):
    """
           Add Gaussian noise to the data
           Args:
               x                   - input vector
               variance_multiplier - coefficient to multiple variance of the noise on
               sigma               - variance of the dataset
           Returns:
               x - output vector, noisy data
    """
    eps = 1e-15
    noise = tf.random_normal(x.shape, 0.0, stddev=np.multiply(sigma, variance_multiplier) + eps)
    x = x + noise
    return x

def loss_reconstruction(output, target, max_vals, pretrain=False):
    """ Reconstruction error. Square of the RMSE

    Args:
      output:    tensor of net output
      target:    tensor of net we are trying to reconstruct
      max_vals:  array of absolute maximal values in the dataset,
                is used for scaling an error to the original space
      pretrain:  wether we are using it during the pretraining phase
    Returns:
      Scalar tensor of mean squared Eucledean distance
    """
    with tf.name_scope("reconstruction_loss"):
        net_output_tf = tf.convert_to_tensor(tf.cast(output, tf.float32), name='input')
        target_tf = tf.convert_to_tensor(tf.cast(target, tf.float32), name='target')

        # Euclidean distance between net_output_tf,target_tf
        error = tf.subtract(net_output_tf, target_tf)

        if not pretrain:
            # Convert it back from the [-1,1] to original values
            error_scaled = tf.multiply(error, max_vals[np.newaxis, :] + 1e-15)
        else:
            error_scaled = error

        squared_error = tf.reduce_mean(tf.square(error_scaled, name="square"), name="averaging")
    return squared_error

class DAE:
    """ Denoising Autoendoder (DAE)

    More details about the network in the original paper:
    http://www.jmlr.org/papers/v11/vincent10a.html

    The user specifies the structure of this network
    by specifying number of inputs, the number of hidden
    units for each layer and the number of final outputs.
    All this information is set in the utils/flags.py file.

    The number of input neurons is defined as a frame_size*chunk_length,
    since it will take a time-window as an input

    """

    def __init__(self, shape, sess, variance_coef, data_info):
        """DAE initializer

        Args:
          shape:          list of ints specifying
                          num input, hidden1 units,...hidden_n units, num outputs
          sess:           tensorflow session object to use
          varience_coef:  multiplicative factor for the variance of noise wrt the variance of data
          data_info:      key information about the dataset
        """

        self.__shape = shape  # [input_dim,hidden1_dim,...,hidden_n_dim,output_dim]
        self.__variables = {}
        self.__sess = sess

        self.num_hidden_layers = np.size(shape) - 2
#         print('self.num_hidden_layers:',self.num_hidden_layers)
        self.batch_size = FLAGS.batch_size
        self.sequence_length = FLAGS.chunk_length

        self.scaling_factor = 1

	    # maximal value and mean pose in the dataset (used for scaling it to interval [-1,1] and back)
        self.max_val = data_info.max_val
        self.mean_pose = data_info.mean_pose


        #################### Add the DATASETS to the GRAPH ###############

        #### 1 - TRAIN ###
        self._train_data_initializer = tf.placeholder(dtype=tf.float32,
                                                      shape=data_info.train_shape)
        self._train_data = tf.Variable(self._train_data_initializer,
                                       trainable=False, collections=[], name='Train_data')
        train_epochs = FLAGS.training_epochs + FLAGS.pretraining_epochs * FLAGS.num_hidden_layers
        train_frames = tf.train.slice_input_producer([self._train_data], num_epochs=train_epochs)
        self._train_batch = tf.train.shuffle_batch(train_frames,
                                                   batch_size=FLAGS.batch_size, capacity=5000,
                                                   min_after_dequeue=1000, name='Train_batch')
        print("data_info.train_shape:", data_info.train_shape)
        print("self._train_data:", self._train_data)
        print("train_epochs:", train_epochs)
        print("train_frames:", train_frames)
        
        #### 2 - VALIDATE, can be used as TEST ###
        # When optimizing - this dataset stores as a validation dataset,
        # when testing - this dataset stores a test dataset
        self._valid_data_initializer = tf.placeholder(dtype=tf.float32,
                                                      shape=data_info.eval_shape)
        self._valid_data = tf.Variable(self._valid_data_initializer,
                                       trainable=False, collections=[], name='Valid_data')
        valid_frames = tf.train.slice_input_producer([self._valid_data],
                                                     num_epochs=FLAGS.training_epochs)
        self._valid_batch = tf.train.shuffle_batch(valid_frames,
                                                   batch_size=FLAGS.batch_size, capacity=5000,
                                                   min_after_dequeue=1000, name='Valid_batch')

        if FLAGS.weight_decay is not None:
            print('\nWe apply weight decay')

        ### Specify tensorflow setup  ###
        with sess.graph.as_default():

            ##############        SETUP VARIABLES       ######################

            with tf.variable_scope("AE_Variables"):

                for i in range(self.num_hidden_layers + 1):  # go over layers

                    # create variables for matrices and biases for each layer
                    self._create_variables(i, FLAGS.weight_decay)

                ##############        DEFINE THE NETWORK     ##################

                ''' 1 - Setup network for TRAINing '''
                # Input noisy data and reconstruct the original one
                # as in Denoising AutoEncoder                
                self._input_ = add_noise(self._train_batch, variance_coef, data_info.data_sigma)
                self._target_ = self._train_batch
                # Define output and loss for the training data
                self._output, _, _ = self.construct_graph(self._input_, FLAGS.dropout)
                self._reconstruction_loss = loss_reconstruction(self._output,
                                                                self._target_, self.max_val)
                tf.add_to_collection('losses', self._reconstruction_loss)  # add weight decay loses
                self._loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

                ''' 2 - Setup network for TESTing '''
                self._valid_input_ = self._valid_batch
                self._valid_target_ = self._valid_batch

                # Define output (no dropout)
                self._valid_output, self._encode, self._decode = \
                    self.construct_graph(self._valid_input_, 1)

                # Define loss
                self._valid_loss = loss_reconstruction(self._valid_output,
                                                       self._valid_target_, self.max_val)
    @property
    def session(self):
        """ Interface for the session"""
        return self.__sess

    @property
    def shape(self):
        """ Interface for the shape"""
        return self.__shape

    # Make more comfortable interface to the network weights

    def _w(self, n, suffix=""):
        return self["matrix"+str(n)+suffix]

    def _b(self, n, suffix=""):
        return self["bias"+str(n)+suffix]

    @staticmethod
    def _feedforward(x, w, b):
        """
        Traditional feedforward layer: multiply on weight matrix, add bias vector
         and apply activation function

        Args:
            x: input ( usually - batch of vectors)
            w: matrix to be multiplied on
            b: bias to be added

        Returns:
            y: result of applying this feedforward layer
        """

        y = tf.tanh(tf.nn.bias_add(tf.matmul(x, w), b))
        return y

    def construct_graph(self, input_seq_pl, dropout):

        """ Construct a TensorFlow graph for the AutoEncoding network

        Args:
          input_seq_pl:     tf placeholder for input data: size [batch_size, sequence_length * DoF]
          dropout:          how much of the input neurons will be activated, value in range [0,1]
        Returns:
          output:           output tensor: result of running input placeholder through the network
          middle_layer:     tensor which is encoding input placeholder into a representation
          decoding:         tensor which is decoding a representation back into the input vector
        """

        network_input = input_seq_pl
        print("network_input:",network_input)
        print("FLAGS.frame_size:", FLAGS.frame_size)
        curr_layer = tf.reshape(network_input, [self.batch_size,
                                                FLAGS.chunk_length * FLAGS.frame_size])

        numb_layers = self.num_hidden_layers + 1

        with tf.name_scope("Joint_run"):

            # Pass through the network
            for i in range(numb_layers):

                if i == FLAGS.middle_layer:
                    # Save middle layer
                    with tf.name_scope('middle_layer'):
                        middle_layer = tf.identity(curr_layer)

                with tf.name_scope('hidden'+str(i)):

                    # First - Apply Dropout
                    curr_layer = tf.nn.dropout(curr_layer, dropout)

                    w = self._w(i + 1)
                    b = self._b(i + 1)

                    curr_layer = self._feedforward(curr_layer, w, b)

            output = curr_layer

        # Now create a decoding network

        with tf.name_scope("Decoding"):

            layer = self._representation = tf.placeholder\
                (dtype=tf.float32, shape=middle_layer.get_shape().as_list(), name="Respres.")

            for i in range(FLAGS.middle_layer, numb_layers):

                with tf.name_scope('hidden' + str(i)):

                    # First - Apply Dropout
                    layer = tf.nn.dropout(layer, dropout)

                    w = self._w(i + 1)
                    b = self._b(i + 1)

                    layer = self._feedforward(layer, w, b)

            decoding = layer

        return output, middle_layer, decoding

    def __getitem__(self, item):
        """Get AutoEncoder tf variable

        Returns the specified variable created by this object.
        Names are weights#, biases#, biases#_out, weights#_fixed,
        biases#_fixed.

        Args:
         item: string, variables internal name
        Returns:
         Tensorflow variable
        """
        return self.__variables[item]

    def __setitem__(self, key, value):
        """Store a TensorFlow variable

        NOTE: Don't call this explicitly. It should
        be used only internally when setting up
        variables.

        Args:
          key: string, name of variable
          value: tensorflow variable
        """
        self.__variables[key] = value

    def _create_variables(self, i, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if 'wd' is specified.
        If 'wd' is None, weight decay is not added for this Variable.

        This function was taken from the web

        Args:
          i: number of hidden layer
          wd: add L2Loss weight decay multiplied by this float.
        Returns:
          Nothing
        """

        # Initialize Train weights
        w_shape = (self.__shape[i], self.__shape[i + 1])
        a = tf.multiply(2.0, tf.sqrt(6.0 / (w_shape[0] + w_shape[1])))
        name_w = "matrix"+str(i + 1)
        self[name_w] = tf.get_variable("Variables/"+name_w,
                                       initializer=tf.random_uniform(w_shape, -1 * a, a))

        # Add weight to the loss function for weight decay
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(self[name_w]), wd, name='wgt_'+str(i)+'_loss')
            tf.add_to_collection('losses', weight_decay)

        # Add the histogram summary
        tf.summary.histogram(name_w, self[name_w])

        # Initialize Train biases
        name_b = "bias"+str(i + 1)
        b_shape = (self.__shape[i + 1],)
        self[name_b] = tf.get_variable("Variables/"+name_b, initializer=tf.zeros(b_shape))

        if i < self.num_hidden_layers:
            # Hidden layer pretrained weights
            # which are used after pretraining before fine-tuning
            self[name_w + "_pretr"] = tf.get_variable(name="Var/" + name_w + "_pretr", initializer=
                                                      tf.random_uniform(w_shape, -1 * a, a),
                                                      trainable=False)
            # Hidden layer pretrained biases
            self[name_b + "_pretr"] = tf.get_variable("Var/"+name_b+"_pretr", trainable=False,
                                                      initializer=tf.zeros(b_shape))

            # Pretraining output training biases
            name_b_out = "bias" + str(i+1) + "_out"
            b_shape = (self.__shape[i],)
            b_init = tf.zeros(b_shape)
            self[name_b_out] = tf.get_variable(name="Var/"+name_b_out, initializer=b_init,
                                               trainable=True)

    def run_less_layers(self, input_pl, n, is_target=False):
        """Return result of a net after n layers or n-1 layer (if is_target is true)
           This function will be used for the layer-wise pretraining of the AE
        Args:
          input_pl:  TensorFlow placeholder of AE inputs
          n:         int specifying pretrain step
          is_target: bool specifying if required tensor
                      should be the target tensor
                     meaning if we should run n layers or n-1 (if is_target)
        Returns:
          Tensor giving pretraining net result or pretraining target
        """
        assert n > 0
        assert n <= self.num_hidden_layers

        last_output = input_pl

        for i in range(n - 1):
            w = self._w(i + 1, "_pretrained")
            b = self._b(i + 1, "_pretrained")

            last_output = self._feedforward(last_output, w, b)

        if is_target:
            return last_output

        last_output = self._feedforward(last_output, self._w(n), self._b(n))

        out = self._feedforward(last_output, self._w(n), self["bias" + str(n) + "_out"])

        return out


# motion_repr_learning/ae/utils/utils.py

In [4]:
""" Dataset class"""

class DataSet(object):
    '''
    A class for storing a dataset and all important information,
    which might be needed during training,
    such as batch size amount of epochs completed and so on.
    '''


    def __init__(self, sequences, batch_size):
        self._batch_size = batch_size
        self._sequences = sequences  # all the sequnces in the dataset
        self._num_sequences = sequences.shape[0]
        self._epochs_completed = 0
        self._index_in_epoch = 0

    # Make interface to the protected variables
    @property
    def sequences(self):
        return self._sequences

    @property
    def num_sequences(self):
        return self._num_sequences

class DataSets(object):
    '''
      A class for storing Train and Eval datasets and all related information,
      '''
    pass


In [5]:
def prepare_motion_data(data_dir): #preprocessing
    """
    Read and preprocess the motion dataset

    Args:
        data_dir:           a directory with the dataset
    Return:
        Y_train:            an array of the training dataset
        Y_train_normalized: training dataset normalized to the values [-1,1]
        Y_test:             an array of the test dataset
        Y_test_normalized:  test dataset normalized to the values [-1,1]
        Y_dev_normalized:   dev dataset normalized to the values [-1,1]
        max_val:            maximal values in the dataset
        mean_pose:          mean pose of the dataset
    """

    # Get the data

    Y_train = np.load(data_dir + '/Y_train.npy')
    Y_dev = np.load(data_dir + '/Y_dev.npy')
    Y_test = np.load(data_dir + '/Y_test.npy')

    # Normalize dataset
    max_val = np.amax(np.absolute(Y_train), axis=(0))
    mean_pose = Y_train.mean(axis=(0))

    Y_train_centered = Y_train - mean_pose[np.newaxis, :]
    Y_dev_centered = Y_dev - mean_pose[np.newaxis, :]
    Y_test_centered = Y_test - mean_pose[np.newaxis, :]

    # Scales all values in the input_data to be between -1 and 1
    eps = 1e-8
    Y_train_normalized = np.divide(Y_train_centered, max_val[np.newaxis, :] + eps)
    Y_dev_normalized = np.divide(Y_dev_centered, max_val[np.newaxis, :] + eps)
    Y_test_normalized = np.divide(Y_test_centered, max_val[np.newaxis, :] + eps)

    # Reshape to accomodate multiple frames at each input

    if FLAGS.chunk_length > 1:
        Y_train_normalized = reshape_dataset(Y_train_normalized)
        Y_dev_normalized = reshape_dataset(Y_dev_normalized)
        Y_test_normalized = reshape_dataset(Y_test_normalized)

    # Pad max values and the mean pose, if neeeded
    if FLAGS.chunk_length > 1:
        max_val = np.tile(max_val, FLAGS.chunk_length)
        mean_pose = np.tile(mean_pose, FLAGS.chunk_length)


#     Some tests for flags
    if FLAGS.restore and FLAGS.pretrain:
        print('ERROR! You cannot restore and pretrain at the same time!'
              ' Please, chose one of these options')
        exit(1)

    if FLAGS.middle_layer > FLAGS.num_hidden_layers:
        print('ERROR! Middle layer cannot be more than number of hidden layers!'
              ' Please, update flags')
        exit(1)

#     Y_dev_normalized = 0
#     Y_test_normalized = 0
#     Y_test = 0
    return Y_train_normalized, Y_train, Y_test_normalized, Y_test,\
           Y_dev_normalized, max_val, mean_pose


In [6]:
def create_nn(train_data, dev_data, max_val, mean_pose, restoring):
    """
    Train or restore a neural network
    Args:
     train_data:         training dataset normalized to the values [-1,1]
     dev_data:           dev dataset normalized to the values [-1,1]
     max_val:            maximal values in the dataset
     mean_pose:          mean pose of the dataset
     restoring:          weather  we are going to just restore already trained model
    Returns:
     nn: neural network, which is ready to use
    """

    # Create DataSet object

    data = DataSets()

    data.train = DataSet(train_data, FLAGS.batch_size)
    data.test = DataSet(dev_data, FLAGS.batch_size)

    # Assign variance
    data.train.sigma = np.std(train_data, axis=(0, 1))

    # Create information about the dataset
    data_info = DataInfo(data.train.sigma, data.train._sequences.shape,
                            data.test._sequences.shape, max_val, mean_pose)


    # Set "restore" flag
    FLAGS.restore = restoring
    
#     self.data_sigma = data_sigma
#     self.train_shape = train_shape
#     self.eval_shape = eval_shape
#     self.max_val = max_val
#     self.mean_pose = mean_pose

    
#     print("data.train:", data.train)
#     print("data_info.data_sigma:", data_info.data_sigma)
#     print("data_info.train_shape:", data_info.train_shape)
#     print("data_info.eval_shape:", data_info.eval_shape)
    
    # Train the network
    nn = learning(data, data_info, just_restore=restoring)

    return nn

In [7]:
def convert_back_to_3D(sequence, max_val, mean_pose):
    '''
    Convert back from the normalized values between -1 and 1 to original 3d coordinates
    and unroll them into the sequence

    Args:
        sequence: sequence of the normalized values
        max_val: maximal value in the dataset
        mean_pose: mean value in the dataset

    Return:
        3d coordinates corresponding to the batch
    '''

    # Convert it back from the [-1,1] to original values
#     reconstructed = np.multiply(sequence, max_val[np.newaxis, :] + 1e-15)
    reconstructed = np.multiply(sequence, max_val[np.newaxis, :] + 1e-8)

    # Add the mean pose back
    reconstructed = reconstructed + mean_pose[np.newaxis, :]

    # Unroll batches into the sequence
    reconstructed = reconstructed.reshape(-1, reconstructed.shape[-1])
    return reconstructed

In [8]:
def loss_reconstruction(output, target, max_vals, pretrain=False):
    """ Reconstruction error. Square of the RMSE

    Args:
      output:    tensor of net output
      target:    tensor of net we are trying to reconstruct
      max_vals:  array of absolute maximal values in the dataset,
                is used for scaling an error to the original space
      pretrain:  wether we are using it during the pretraining phase
    Returns:
      Scalar tensor of mean squared Eucledean distance
    """
    with tf.name_scope("reconstruction_loss"):
        net_output_tf = tf.convert_to_tensor(tf.cast(output, tf.float32), name='input')
        target_tf = tf.convert_to_tensor(tf.cast(target, tf.float32), name='target')

        # Euclidean distance between net_output_tf,target_tf
        error = tf.subtract(net_output_tf, target_tf)

        if not pretrain:
            # Convert it back from the [-1,1] to original values
            error_scaled = tf.multiply(error, max_vals[np.newaxis, :] + 1e-15)
        else:
            error_scaled = error

        squared_error = tf.reduce_mean(tf.square(error_scaled, name="square"), name="averaging")
    return squared_error



In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
import sys
from tensorflow.keras.models import load_model
import numpy as np

# predict.py

In [11]:
def predict(model_name, input_file, output_file):
    """ Predict human gesture based on the speech

    Args:
        model_name:  name of the Keras model to be used
        input_file:  file name of the audio input
        output_file: file name for the gesture output

    Returns:

    """
    model = load_model(model_name)
    X = np.load(input_file)

    predicted = np.array(model.predict(X))
    print(predicted.shape)
    np.savetxt(output_file, predicted)




In [13]:
model_name = 'first_model_3d_180_dim'
input_file = 'data_3d_cord/test_inputs/X_test_audio42.npy'
output_file = 'data_3d_cord/test_inputs/predict_audio42_20fps_3d_cord.txt'
predict(model_name, input_file, output_file)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(1218, 180)


# motion_repr_learning/ae/decode.py

In [14]:
DATA_DIR = 'data_3d_cord'
TEST_FILE = 'data_3d_cord/test_inputs/predict_audio42_20fps_3d_cord.txt'
OUTPUT_FILE = 'data_3d_cord/test_inputs/predict_motion42_20fps_3d_cord.txt'


# Get the data
Y_train_normalized, Y_train, Y_test_normalized, Y_test, Y_dev_normalized, max_val, mean_pose  = prepare_motion_data(DATA_DIR)

# Train the network
nn = create_nn(Y_train_normalized, Y_dev_normalized, max_val, mean_pose, restoring=True)

# # Read the encoding
# encoding = np.loadtxt(TEST_FILE)

# print(encoding.shape)

# print("encoding", encoding)

# # Decode it
# decoding = decode(nn, encoding)

# print(decoding.shape)

# np.savetxt(OUTPUT_FILE, decoding, delimiter = ' ')

# # Close Tf session
# nn.session.close()


hidden_shapes: [180]
FLAGS.frame_size: 204
shape: [204, 180, 204]
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [180] rhs shape= [90]
	 [[node Train/save/Assign_1 (defined at C:\Users\Wen\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'Train/save/Assign_1':
  File "C:\Users\Wen\miniconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Wen\miniconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Wen\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Wen\miniconda3\envs\tensorflow\lib\site-packages\traitlets\config\applic

In [ ]:
decoding

In [15]:
decoding.shape

(1152, 204)